In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
def read_data(drunkfile, normalfile):
    drunk = pd.read_csv(drunkfile)
    normal = pd.read_csv(normalfile)
    data = pd.concat([drunk, normal])
    ans_list = [1]*len(drunk) + [0]*len(normal)
    return data[['smile','anger','contempt','disgust','fear','happiness','neutral','sadness','surprise','eyeOccluded']].values, ans_list

In [3]:
X_train, Y_train = read_data("drunk.csv", "normal.csv")

In [4]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [5]:
X_train[X_train==True] = 1
X_train[X_train==False] = 0

In [6]:
(X_train, Y_train) = shuffle(X_train, Y_train)

In [7]:
X_train, X_valid = X_train[:-50], X_train[-50:]
Y_train, Y_valid = Y_train[:-50], Y_train[-50:]

In [8]:
model = Sequential()
model.add(Dense(input_dim = X_train.shape[1], units = 10, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 8, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
# callbacks = []
# callbacks.append(ModelCheckpoint('models/model-{epoch:05d}-{val_acc:.5f}-{val_loss:.5f}.npy', monitor='val_acc', save_best_only=True, mode = 'auto', period=1))
# model.fit(X_train, Y_train, batch_size = 8, epochs = 100, validation_split=0.2, callbacks=callbacks)
model.fit(X_train, Y_train, batch_size = 8, epochs = 25, validation_split=0.2)
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

Train on 1018 samples, validate on 255 samples
Epoch 1/25
1018/1018 [==============================] - 1s 624us/step - loss: 0.6232 - accuracy: 0.6444 - val_loss: 0.5810 - val_accuracy: 0.6706
Epoch 2/25
1018/1018 [==============================] - 0s 222us/step - loss: 0.6228 - accuracy: 0.6434 - val_loss: 0.5750 - val_accuracy: 0.6706
Epoch 3/25
1018/1018 [==============================] - 0s 221us/step - loss: 0.6177 - accuracy: 0.6434 - val_loss: 0.5827 - val_accuracy: 0.6706
Epoch 4/25
1018/1018 [==============================] - 0s 213us/step - loss: 0.6139 - accuracy: 0.6434 - val_loss: 0.5812 - val_accuracy: 0.6706
Epoch 5/25
1018/1018 [==============================] - 0s 221us/step - loss: 0.6023 - accuracy: 0.6473 - val_loss: 0.5565 - val_accuracy: 0.6706
Epoch 6/25
1018/1018 [==============================] - 0s 220us/step - loss: 0.5878 - accuracy: 0.7141 - val_loss: 0.5389 - val_accuracy: 0.7882
Epoch 7/25
1018/1018 [==============================] - 0s 224us/step - loss:

In [9]:
from xgboost import XGBClassifier
model2 = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.05)
model2.fit(X_train, Y_train.flatten())
predict_train = model2.predict(X_train)
acc_train = np.mean(Y_train.flatten() == predict_train)
print("acc_train = ", acc_train)
predict_valid = model2.predict(X_valid)
acc_valid = np.mean(Y_valid.flatten() == predict_valid)
print("acc_valid = ", acc_valid)

acc_train =  0.8664571877454831
acc_valid =  0.82


In [10]:
model.save("model.h5")

In [11]:
from keras.models import load_model
model = load_model("model.h5")
result = model.evaluate(X_valid, Y_valid, batch_size = X_valid.shape[0])
print(result[1])

50/50 [==============================] - 0s 777us/step
0.8199999928474426


In [12]:
ans = model.predict(np.array([[0.001,0,0.84,0.021,0.001,0.001,0.064,0.073,0,1]]))
print(ans)

[[0.9874679]]


In [13]:
ans = model.predict(np.array([[0.003,0,0.002,0,0,0.003,0.935,0,0,0]]))
print(ans)

[[0.22729188]]
